# Using enterprise to analyze PTA data

In this notebook you will learn:
* How to use `enterprise` to interact with IPTA data,
* How to search in PTA data for GWs,
* How to perform Bayesian model selection,
* How to post-process your results.

# Load modules

In [ ]:
from __future__ import division

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

import os, glob, json, pickle
import matplotlib.pyplot as plt
import numpy as np
import scipy.linalg as sl

import enterprise
from enterprise.pulsar import Pulsar
import enterprise.signals.parameter as parameter
from enterprise.signals import utils
from enterprise.signals import signal_base
from enterprise.signals import selections
from enterprise.signals.selections import Selection
from enterprise.signals import white_signals
from enterprise.signals import gp_signals
from enterprise.signals import deterministic_signals
import enterprise.constants as const

import corner
from PTMCMCSampler.PTMCMCSampler import PTSampler as ptmcmc

## Get par, tim, and noise files (this is not the preferred method when we have supplied pickled enterprise Pulsar files; see below)
Here we collect the tim and par files. 

In [ ]:
psrlist = None # define a list of pulsar name strings that can be used to filter.

In [ ]:
datadir = '/Users/taylosr8/Research/NANOGrav/NANOGrav_12y/' # set your data directory

In [ ]:
parfiles = sorted(glob.glob(datadir + 'par/*.par'))
timfiles = sorted(glob.glob(datadir + 'tim/*.tim'))

# filter
if psrlist is not None:
    parfiles = [x for x in parfiles if x.split('/')[-1].split('.')[0] in psrlist]
    timfiles = [x for x in timfiles if x.split('/')[-1].split('.')[0] in psrlist]   
    
# Make sure you use the tempo2 parfile for J1713+0747!!
# ...filtering out the tempo parfile... 
parfiles = [x for x in parfiles if 'J1713+0747_NANOGrav_12yv2.gls.par' not in x]

In [ ]:
len(parfiles)

## Load into Pulsar class list

* The `enterprise` Pulsar class uses `libstempo` to read in `par` and `tim` files, then stores all pulsar data into a `Pulsar` object. This object contains all data and meta-data needed for the ensuing pulsar and PTA analysis. You no longer to reference the `par` and `tim` files after this cell.
* Note below that you can explicitly declare which version of the JPL solar-system ephemeris model that will be used to compute the Roemer delay between the geocenter and the barycenter (e.g. `DE438`). Otherwise the default values will be taken from the `par` files. Explicitly declaring the version here is good practice.
* You can also explicitly set the clock file to a version of `BIPM`, e.g. `BIPM(2018)`. This is less important, and you can let the code take the value from the `par` file.
* When you execute the following cell, you will get warnings like `WARNING: Could not find pulsar distance for PSR ...`. Don't worry! This is expected, and fine. Not all pulsars have well constrained distances, and will be set to `1 kpc` with a `20%` uncertainty.

### Read par and tim files into enterprise Pulsar objects

In [ ]:
psrs = []
for p, t in zip(parfiles, timfiles):
    psr = Pulsar(p, t, ephem='DE438', clk='BIPM(2018)')
    psrs.append(psr)

## OR... load in enterprise pickled Pulsar instances that we've prepared!
Go here for full details: https://paper.dropbox.com/doc/NG-12.5yr_v3-GWB-Analysis--A2zJbxQU704Oq9jU1oqAWJCHAQ-DICJei6NxsPjxnO90mGMo

Pickled 12.5yr pulsars: https://drive.google.com/file/d/1GUcmdj9OMf7-hrAOydeHEO4ylQxgC9Kb/edit

Noise files: https://drive.google.com/file/d/1V7bu2y5hxFSj_7KWO3uNM7Q_0dciXfSX/edit

Empirical red noise proposal distributions: https://drive.google.com/file/d/19odsqZ93Wh8og1AGdU7SfvXzB0DJwipG/edit

In [ ]:
## set your data directory
datadir = '/Users/taylosr8/Downloads/' # set your data directory

## read in pickles
psrs = pickle.load(open(datadir + 'channelized_12yr_v3_partim_DE438.pkl', 'rb'))

* We can read-in some previously computed noise properties from single-pulsar analyses. These are things like `EFAC`, `EQUAD`, and (for `NANOGrav`) `ECORR`. 
* In practice, we set these white-noise properties as fixed in the low-frequency noise / GW searches.
* The noise properties have been stored as `json` files, and are read in to a big parameter dictionary.

In [ ]:
## Get parameter noise dictionary
noise_ng12 = datadir + 'channelized_12p5yr_v3_full_noisedict.json'

params = {}
with open(noise_ng12, 'r') as fp:
    params.update(json.load(fp))

In [ ]:
## Load in empirical distributions
#emp_dists = pickle.load(open(datadir + '12yr_emp_dist_RNonly_py3.pkl', 'rb'))
#emp_dists = '/home/stephen.taylor/NANOGrav/nanograv_12p5yr_analysis/nanograv_12p5yr_analysis_mar2020/data/12yr_emp_dist_RNonly_py3.pkl'

# PTA Parameter Estimation

## Set up `enterprise` model for PTA upper-limit (*verbose version*)

* Usually, in a full PTA analysis we fix all of the white noise (EFAC, EQUAD, and ECORR) parameters to the values obtained from the noise files. This is done by using `Constant` parameters. In this case we do not specify a default value for all instances of that parameter but instead will set them, based on their initialized pulsar and backend specific name, later via the `set_default_params` method of `PTA`. 

* We use the `Selection` object to define which noise parameters are assigned to which chunks of TOAs. This selection is based on unique combination of backends and receivers.

* Another feature to notice is that **for upper limits** we do not use a uniform prior on the log of the red-noise or GWB amplitude. Instead we use a `LinearExp` prior (short for linear-exponent prior), that is a prior of the form $p(x)\propto 10^x$. This is how we can still use the log of the parameter to sample but place a uniform prior on the parameter itself. We do this for both the red noise and GWB amplitude parameters. **For detection analyses** we still use a `Uniform` prior on the log of the red-noise and GWB amplitude. 

* In order to save on computing time we do not include spatial correlations here. Instead we model the GWB as a common red process across all pulsars. In `enterprise` we can do this with a simple trick. We pre-initialize the parameters before passing them to the `Signal` model. In this way the *same* parameter instance is used for all pulsars. Lastly, we fixt the spectral index of the GWB to be 13/3 (4.33) using the `Constant` parameter.

In [ ]:
# find the maximum time span to set GW frequency sampling
tmin = [p.toas.min() for p in psrs]
tmax = [p.toas.max() for p in psrs]
Tspan = np.max(tmax) - np.min(tmin)

In [ ]:
# define selection by observing backend
selection = selections.Selection(selections.by_backend)

### Parameters

In [ ]:
# white noise parameters
# since we are fixing these to values from the noise file we set
# them as constant parameters
efac = parameter.Constant()
equad = parameter.Constant()
ecorr = parameter.Constant()

# red noise parameters
log10_A = parameter.LinearExp(-20, -11)
gamma = parameter.Uniform(0, 7)

# dm-variation parameters
log10_A_dm = parameter.LinearExp(-20, -11)
gamma_dm = parameter.Uniform(0, 7)

# GW parameters (initialize with names here to use parameters in common across pulsars)
log10_A_gw = parameter.LinearExp(-18,-12)('log10_A_gw')
gamma_gw = parameter.Constant(4.33)('gamma_gw')

### Signals

In [ ]:
# white noise
ef = white_signals.MeasurementNoise(efac=efac, selection=selection)
eq = white_signals.EquadNoise(log10_equad=equad, selection=selection)
ec = white_signals.EcorrKernelNoise(log10_ecorr=ecorr, selection=selection2)

# red noise (powerlaw with 30 frequencies)
pl = utils.powerlaw(log10_A=log10_A, gamma=gamma)
rn = gp_signals.FourierBasisGP(spectrum=pl, components=30, Tspan=Tspan)

# gwb (no spatial correlations)
cpl = utils.powerlaw(log10_A=log10_A_gw, gamma=gamma_gw)
gw = gp_signals.FourierBasisGP(spectrum=cpl, components=30, Tspan=Tspan, name='gw')

# for spatial correltions you can do...
#orf = utils.hd_orf()
#crn = gp_signals.FourierBasisCommonGP(cpl, orf, components=30, Tspan=Tspan, name='gw')

# to add solar system ephemeris modeling...
bayesephem=False
if bayesephem:
    eph = deterministic_signals.PhysicalEphemerisSignal(use_epoch_toas=True)

# timing model
tm = gp_signals.TimingModel(use_svd=True)

In [ ]:
# full model
if bayesephem:
    s = ef + eq + rn + dm_gp + tm + eph + gw
else:
    s = ef + eq + rn + dm_gp + tm + gw

In [ ]:
# intialize PTA
models = []
        
for p in psrs:    
    models.append(s(p))
    
pta = signal_base.PTA(models)

In [ ]:
pta.params

### Set white noise parameters

In [ ]:
pta.set_default_params(params)

### Set initial parameters drawn from prior

In [ ]:
x0 = np.hstack([p.sample() for p in pta.params])
ndim = len(x0)

### Set up sampler

In [ ]:
# initial jump covariance matrix
cov = np.diag(np.ones(ndim) * 0.01**2)

sampler = ptmcmc(ndim, pta.get_lnlikelihood, pta.get_lnprior, cov, 
                 outDir='./chains_pta_test/', resume=False)

### Sample!

In [ ]:
# sampler for N steps
N = int(5e6)
x0 = np.hstack(p.sample() for p in pta.params)
sampler.sample(x0, N, SCAMweight=30, AMweight=15, DEweight=50, )

### Plot output

In [ ]:
chain = np.loadtxt('./chains_pta_test/chain_1.txt')
burn = int(0.25 * chain.shape[0])

In [ ]:
plt.hist(chain[burn:,-5], 50, normed=True, histtype='step', lw=2);

### Upper limit value

In [ ]:
upper = 10**np.percentile(chain[burn:, -5], q=95)
print(upper)

## Now, the easy way to do all of this...

We use `enterprise_extensions` as in the single-pulsar analysis tutorial.

In [ ]:
import enterprise_extensions
from enterprise_extensions import models, model_utils, hypermodel

In [ ]:
pta = models.model_general(psrs, red_psd='powerlaw', common_psd='powerlaw', 
                           red_components=30, common_components=30, 
                           white_vary=False, noisedict=params, tm_svd=True, 
                           gamma_common=4.33, upper_limit_red=True, 
                           upper_limit_common=True, bayesephem=True)
# note that bayesephem is switched on here, but you can also switch it off.

In [ ]:
# Setup an instance of a HyperModel.
# This doesn't mean we are doing model selection (yet!), but the 
# hypermodel module gives access to some nifty sampling schemes.
super_model = hypermodel.HyperModel({0: pta})

In [ ]:
outdir = './chains_pta_test/'
sampler = super_model.setup_sampler(resume=False, outdir=outdir, sample_nmodel=False)

In [ ]:
# sampler for N steps
N = int(5e6)
x0 = super_model.initial_sample()

In [ ]:
# sample
sampler.sample(x0, N, SCAMweight=30, AMweight=15, DEweight=50, )

In [ ]:
# Read in chains and parameters

chain = np.loadtxt(outdir + '/chain_1.txt')
burn = int(0.25*chain.shape[0])
pars = np.loadtxt(outdir + '/pars.txt', dtype=np.unicode_)

pp = model_utils.PostProcessing(chain, pars)

In [ ]:
# Plot GW amplitude posterior
ind = list(pars).index('log10_A_gw')
plt.hist(chain[burn:,ind], bins=40);

In [ ]:
# Compute upper limit
print model_utils.ul(chain[burn:, ind], q=95.0)

# PTA Model Selection

We want to be able to compute the Bayesian odds for a GWB in the data. This can be done using the hypermodel class, where we choose between a model with a common (but uncorrelated) red process in the pulsars, and a GWB affecting all pulsars.

We typically perform detection-type analyses with uniform-in-log priors on all amplitude parameters for low-frequency processes. This is implemented below whenever we switch `upper_limit` to be equal to `False`.

## Setup dictionary of PTA models

In [ ]:
nmodels = 2
mod_index = np.arange(nmodels)

# Make dictionary of PTAs.
pta = dict.fromkeys(mod_index)
pta[0] = models.model_general(psrs, red_psd='powerlaw', common_psd='powerlaw', 
                           red_components=30, common_components=30, 
                           white_vary=False, noisedict=params, tm_svd=True, 
                           gamma_common=4.33, upper_limit_red=False, 
                           upper_limit_common=False, bayesephem=True, orf=None)
pta[1] = models.model_general(psrs, red_psd='powerlaw', common_psd='powerlaw', 
                           red_components=30, common_components=30, 
                           white_vary=False, noisedict=params, tm_svd=True, 
                           gamma_common=4.33, upper_limit_red=False, 
                           upper_limit_common=False, bayesephem=True, orf='hd')

In [ ]:
super_model = model_utils.HyperModel(pta)

## Sample

In [ ]:
sampler = super_model.setup_sampler(resume=False, outdir=outdir, sample_nmodel=True)

In [ ]:
# sampler for N steps
N = int(5e6)
x0 = super_model.initial_sample()

In [ ]:
# sample
sampler.sample(x0, N, SCAMweight=30, AMweight=15, DEweight=50, )

## Post-process

In [ ]:
chain = np.loadtxt(outdir + '/chain_1.txt')
burn = int(0.25*chain.shape[0])
pars = np.loadtxt(outdir + '/pars.txt', dtype=np.unicode_)

pp = model_utils.PostProcessing(chain, pars)

In [ ]:
# Plot histgram for GW amplitude
chain_burn = chain[burn:,:]

ind_model = list(pars).index('nmodel')
ind_gwamp = list(pars).index('log10_A_gw')

# ORF = None
#plt.hist(chain_burn[chain_burn[:, ind_model] < 0.5, ind_gwamp], bins=40);

# ORF = Hellings & Downs
plt.hist(chain_burn[chain_burn[:, ind_model] > 0.5, ind_gwamp], bins=40);

In [ ]:
# Plot histogram for GWB model selection
plt.hist(chain_burn[:, ind_model], bins=40);

## Bayes factors

### Savage-Dickey Bayes factor

This gives the signal-vs-noise Bayes factor for a common red process in the pulsars plus intrisnc noise, versus intrinsic noise alone.

In [ ]:
print(model_utils.bayes_fac(chain_burn[chain_burn[:, ind_model] < 0.5, ind_gwamp], ntol=1))

### Posterior odds ratio

This gives the Bayesian odds between a model with a Hellings & Downs correlated red process between pulsars, and a common (but uncorrelated) red process between pulsars. This is the smoking-gun detection statsitic for a GWB signal.

In [ ]:
print(model_utils.odds_ratio(chain_burn[:, ind_model], models=[0,1]))